# Part 3: Deploy a simple ML model with an input and an output 

## Import libraries

In [ ]:
import craft_ai_sdk
from craft_ai_sdk.io import Input, Output
from craft_ai_sdk.io import DeploymentOutputMapping

import dotenv
import os
import requests

## Load environnement variables

In [ ]:
dotenv.load_dotenv()

SDK_TOKEN = os.environ['CRAFT_AI_ACCESS_TOKEN']
ENVIRONMENT_URL = os.environ['CRAFT_AI_ENVIRONMENT_URL']


## SDK instantiation

In [ ]:
sdk = craft_ai_sdk.CraftAiSdk(sdk_token=SDK_TOKEN, environment_url=ENVIRONMENT_URL)

## Step creation with the SDK

### Declare Input and Output of our new step

#### Input

In [ ]:
prediction_input = Input(
    name="input_data", 
    data_type="json"
)

#### Output

In [ ]:
prediction_output = Output(
    name="predictions",
    data_type="json"
)

### Create a step

In [ ]:
sdk.create_step(
    step_name="part-3-irisio-step",
    function_path="src/part-3-iris_ModelIO.py",
    function_name="TrainPredictIris", 
	description="This function creates a classifier model for iris and makes prediction on test data set",
	inputs=[prediction_input],
    outputs=[prediction_output],
)

### List the step

In [ ]:
step_list = sdk.list_steps()
step_list

### Get step information

In [ ]:
step_info = sdk.get_step("part-3-irisio-step")
step_info

## Create a pipeline

### Create a pipeline

In [ ]:
sdk.create_pipeline(
    pipeline_name="part-3-irisio-pipeline",
		step_name="part-3-irisio-step",
)

## Deploy your Pipeline in an Endpoint

In [ ]:
output_mapping = DeploymentOutputMapping(
    step_output_name='predictions', 
    endpoint_output_name='iris_species'
)

### Create endpoint

In [ ]:
endpoint = sdk.create_endpoint(
	pipeline_name="part-3-irisio-pipeline",
	endpoint_name="part-3-irisio-endpoint",
    outputs_mapping=[output_mapping]
)

In [ ]:
endpoint

### Endpoint call

#### Prepare input data

In [ ]:
# prepare input data for which we want predictions
import numpy as np
import pandas as pd 
from sklearn import datasets

np.random.seed(0)
indices = np.random.permutation(150)
iris_X, iris_y = datasets.load_iris(return_X_y=True, as_frame=True)
iris_X_test = iris_X.loc[indices[90:120],:]

# convert our test dataframe into a dictionary as required
test_dict_data = iris_X_test.to_dict(orient="index")

In [ ]:
test_dict_data

In [ ]:
test_data = {
	"input_data": test_dict_data
}

#### Call our endpoint

In [ ]:
endpoint_URL = f"{os.environ.get('CRAFT_AI_ENVIRONMENT_URL')}/endpoints/" + endpoint["name"]
headers = {"Authorization": "EndpointToken " + endpoint["endpoint_token"]}
request = requests.post(endpoint_URL, headers=headers, json=test_data)

request.status_code

## Execution verification

#### Display output

In [ ]:
request.json()['results']

### Display logs

In [ ]:
pipeline_executions = sdk.list_pipeline_executions(pipeline_name="part-3-irisio-pipeline")

In [ ]:
logs = sdk.get_pipeline_execution_logs(pipeline_name="part-3-irisio-pipeline", 
	execution_id=pipeline_executions[-1]['execution_id'])

print('\n'.join(log["message"] for log in logs))